# Analysis Template Walkthrough

# Setup

## Select extract
UCSD is a special case, with data retrieved by manual extract as an Excel spreadsheet

In [1]:
repository = 'ucsd'
object_type = 'records'

In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [3]:
#see more rows and columns of output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100) 

## Helper Functions

In [4]:
import os, sys
dir2 = os.path.abspath('../')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

from utils import analysis
from utils.crosswalk import RepositoryExtract, property_crosswalk
from utils import accessors

# Summary Statistic Walkthroughs

Read in the repository .xlsx file

In [5]:
df = pd.read_excel('UCSD_2020-02-24.xlsx')

In [6]:
df

,Object_Unique_ID,Object_Unique_ID_(ARK),Level,File_name,File_use,Type_of_Resource,Language,Title,Date_collected,Date_creation,Date_issued,Begin_date,End_date,Person_Creator,Person_Principal_investigator,Person_advisor,Person_author,Person_co-principal_investigator,Person_contributor,Person_data_contributor,Person_data_manager,Corporate_contributor,Geographic_point,Identifier__(ORCID),Identifier_DOI,Note_description,Note_funding,Note_local_attribution,Note_note,Note_preferred_citation,Note_scope_and_content,Note_technical_details,Note_related_publications,Related_resource_IsDerivedFrom_12,Related_resource_is_derived_from_-_dataset_12,Related_resource_is_new_version_of_18,Related_resource_is_supplement_to_-_publication_11,Related_resource_references_23,Related_resource_related_25,Related_resource_related,Subject_corporate_name,Subject_geographic,Subject_scientific_name,Subject_series,Subject_topic,collection_s,copyrightJurisdiction,copyrightNote,copyrightPurposeNote,copyrightStatus,license_licenseNote,license_licenseURI,license_permission/type,rightsHolderCorporate,rightsHolderPersonal,file_Name,file_File_format,file_Media_type,file_File_size
0,bb5023335q,bb5023335q,Object,NaN,NaN,data,zxx - No linguistic content; Not applicable @ ...,Data from: A Microfluidic Device for Automated...,NaN,Time period of project: 2018-12-01 to 2020-01-15|,2021.0,2018-12-01,2020-01-15,"Roy, Priyatanu @ bb2770755m","Dutcher, Cari S. @ bb7617216s",NaN,"House, Margaret L. @ bb0211004b|Roy, Priyatanu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.6075/J02N50T8,Publication abstract: Measurement of ice nucle...,This work was supported by NSF through the NSF...,"Research Data Curation Program, UC San Diego, ...",NaN,"Roy, Priyatanu; House, Margaret; Dutcher, Cari...",This package contains an explanatory readme fi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High throughput @ bb3112054r|Ice nucleating pa...,Center for Aerosol Impacts on Chemistry of the...,US,Constraint(s) on Use: This work is protected b...,Use: This work is available from the UC San Di...,Under copyright,Creative Commons Attribution 4.0 International...,http://creativecommons.org/licenses/by/4.0/,display,UC Regents,NaN,NaN,NaN,NaN,NaN
1,bb5023335q,bb5023335q/1,Component,README_v2.txt,data-service,NaN,NaN,Readme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,README_v2.txt,Plain text,\ttext/plain,1772.0
2,bb5023335q,bb5023335q/2,Component,Data.zip,data-service,NaN,NaN,Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\tData.zip,ZIP Format,application/zip,439893640.0
3,bb2666864s,bb2666864s,Object,NaN,NaN,data|text,eng - English @ bb87044465,DSE Capstone - American Gut Project Cohort4 2019,NaN,2019-01-04 - 2019-06-06,2019.0,2019-01-04,2019-06-06,NaN,NaN,"Knight, Rob @ bb6933113m|McDonald, Daniel @ bb...","Conrad, Ryan @ bb9834160g|Rowan, Sean @ bb4680...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.6075/J0HT2MN3,Abstract: \nThe American Gut Project (AGP) [1...,NaN,"Research Data Curation Program, UC San Diego, ...",NaN,"Conrad, Ryan; Inghilterra, Ryan; Rowan, Sean; ...",NaN,NaN,References:\n\nDrugBank 5.0: a major update to...,NaN,NaN,NaN,NaN,NaN,NaN,AGP Paper - https://msystems.asm.org/content/3...,NaN,NaN,NaN,DSE MAS - 2019 Cohort @ bb90491736,Luigi (Python package) @ bb0619060v|Data Scien...,Data Science & Engineering Master of Advanced ...,US,Constraint(s) on Use: This work is protected b...,Use: This work is available from the UC San Di...,Under copyright,Creative Commons Attribution 4.0 International...,http://creativecommons.org/licenses/by/4.0/,display,NaN,"Conrad, Ryan; Inghilterra, Ryan; Rowan, Sean; ...",NaN,NaN,NaN,NaN
4,bb2666864s,bb2666864s/1,Component,README_Group1.md,document-se

In [7]:
#replace NaN values with None
df = df.astype(object).replace(np.nan, None)
df

,Object_Unique_ID,Object_Unique_ID_(ARK),Level,File_name,File_use,Type_of_Resource,Language,Title,Date_collected,Date_creation,Date_issued,Begin_date,End_date,Person_Creator,Person_Principal_investigator,Person_advisor,Person_author,Person_co-principal_investigator,Person_contributor,Person_data_contributor,Person_data_manager,Corporate_contributor,Geographic_point,Identifier__(ORCID),Identifier_DOI,Note_description,Note_funding,Note_local_attribution,Note_note,Note_preferred_citation,Note_scope_and_content,Note_technical_details,Note_related_publications,Related_resource_IsDerivedFrom_12,Related_resource_is_derived_from_-_dataset_12,Related_resource_is_new_version_of_18,Related_resource_is_supplement_to_-_publication_11,Related_resource_references_23,Related_resource_related_25,Related_resource_related,Subject_corporate_name,Subject_geographic,Subject_scientific_name,Subject_series,Subject_topic,collection_s,copyrightJurisdiction,copyrightNote,copyrightPurposeNote,copyrightStatus,license_licenseNote,license_licenseURI,license_permission/type,rightsHolderCorporate,rightsHolderPersonal,file_Name,file_File_format,file_Media_type,file_File_size
0,bb5023335q,bb5023335q,Object,None,None,data,zxx - No linguistic content; Not applicable @ ...,Data from: A Microfluidic Device for Automated...,None,Time period of project: 2018-12-01 to 2020-01-15|,2021.0,2018-12-01,2020-01-15,"Roy, Priyatanu @ bb2770755m","Dutcher, Cari S. @ bb7617216s",None,"House, Margaret L. @ bb0211004b|Roy, Priyatanu...",None,None,None,None,None,None,None,https://doi.org/10.6075/J02N50T8,Publication abstract: Measurement of ice nucle...,This work was supported by NSF through the NSF...,"Research Data Curation Program, UC San Diego, ...",None,"Roy, Priyatanu; House, Margaret; Dutcher, Cari...",This package contains an explanatory readme fi...,None,None,None,None,None,None,None,None,None,None,None,None,None,High throughput @ bb3112054r|Ice nucleating pa...,Center for Aerosol Impacts on Chemistry of the...,US,Constraint(s) on Use: This work is protected b...,Use: This work is available from the UC San Di...,Under copyright,Creative Commons Attribution 4.0 International...,http://creativecommons.org/licenses/by/4.0/,display,UC Regents,None,None,None,None,None
1,bb5023335q,bb5023335q/1,Component,README_v2.txt,data-service,None,None,Readme,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,README_v2.txt,Plain text,\ttext/plain,1772.0
2,bb5023335q,bb5023335q/2,Component,Data.zip,data-service,None,None,Data,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,\tData.zip,ZIP Format,application/zip,439893640.0
3,bb2666864s,bb2666864s,Object,None,None,data|text,eng - English @ bb87044465,DSE Capstone - American Gut Project Cohort4 2019,None,2019-01-04 - 2019-06-06,2019.0,2019-01-04,2019-06-06,None,None,"Knight, Rob @ bb6933113m|McDonald, Daniel @ bb...","Conrad, Ryan @ bb9834160g|Rowan, Sean @ bb4680...",None,None,None,None,None,None,None,https://doi.org/10.6075/J0HT2MN3,Abstract: \nThe American Gut Project (AGP) [1...,None,"Research Data Curation Program, UC San Diego, ...",None,"Conrad, Ryan; Inghilterra, Ryan; Rowan, Sean; ...",None,None,References:\n\nDrugBank 5.0: a major update to...,None,None,None,None,None,None,AGP Paper - https://msystems.asm.org/content/3...,None,None,None,DSE MAS - 2019 Cohort @ bb90491736,Luigi (Python package) @ bb0619060v|Data Scien...,Data Science & Engineering Master of Advanced ...,US,Constraint(s) on Use: This work is protected b...,Use: This work is available from the UC San Di...,Under copyright,Creative Commons Attribution 4.0 International...,http://creativecommons.org/licenses

### Portions done manually - related resources

Components need to be grouped to object level to get object level results
Note also that columns aren't already lists when they need to be for analysis

In [8]:
df.columns

Index(['Object_Unique_ID', 'Object_Unique_ID_(ARK)', 'Level', 'File_name',
       'File_use', 'Type_of_Resource', 'Language', 'Title', 'Date_collected',
       'Date_creation', 'Date_issued', 'Begin_date', 'End_date',
       'Person_Creator', 'Person_Principal_investigator', 'Person_advisor',
       'Person_author', 'Person_co-principal_investigator',
       'Person_contributor', 'Person_data_contributor', 'Person_data_manager',
       'Corporate_contributor', 'Geographic_point', 'Identifier__(ORCID)',
       'Identifier_DOI', 'Note_description', 'Note_funding',
       'Note_local_attribution', 'Note_note', 'Note_preferred_citation',
       'Note_scope_and_content', 'Note_technical_details',
       'Note_related_publications', 'Related_resource_IsDerivedFrom_12',
       'Related_resource_is_derived_from_-_dataset_12',
       'Related_resource_is_new_version_of_18',
       'Related_resource_is_supplement_to_-_publication_11',
       'Related_resource_references_23', 'Related_resource_re

## 1. How many total objects (not just records) are in our main dataset extracts for each repository?
**Property:** unique_identifier

In [9]:
ids = df.UCSDRecordsCrosswalk.unique_identifier
ids

0      bb5023335q
1      bb5023335q
2      bb5023335q
3      bb2666864s
4      bb2666864s
          ...    
169    bb4749622c
170    bb4749622c
171    bb4749622c
172    bb4749622c
173    bb4749622c
Name: Object_Unique_ID, Length: 174, dtype: object

In [10]:
ids.nunique()

25

In [11]:
print(f'There are {len(ids)} items in the UCSD extract, with {ids.nunique()} unique IDs.')

There are 174 items in the UCSD extract, with 25 unique IDs.


## 2. See the "Licenses offered" tab in /Working documents/Licenses sheet for list of licenses by repo.

## Given the type(s) of license(s) offered by the repo, how many of each type is assigned?
**Property:** License

In [12]:
licenses = df.UCSDRecordsCrosswalk.license
licenses

0      http://creativecommons.org/licenses/by/4.0/
1                                             None
2                                             None
3      http://creativecommons.org/licenses/by/4.0/
4                                             None
                          ...                     
169                                           None
170                                           None
171                                           None
172                                           None
173                                           None
Name: license_licenseURI, Length: 174, dtype: object

In [13]:
#licenses are assigned at object level
#since interested in per object, need to group by ID
license_ids = pd.concat([ids, licenses], axis = 1)
license_ids

,Object_Unique_ID,license_licenseURI
0,bb5023335q,http://creativecommons.org/licenses/by/4.0/
1,bb5023335q,None
2,bb5023335q,None
3,bb2666864s,http://creativecommons.org/licenses/by/4.0/
4,bb2666864s,None
...,...,...
169,bb4749622c,None
170,bb4749622c,None
171,bb4749622c,None
172,bb4749622c,None


In [14]:
#drop duplicates and remove none
license_ids = license_ids.drop_duplicates()
license_ids = license_ids.dropna()
license_ids

,Object_Unique_ID,license_licenseURI
0,bb5023335q,http://creativecommons.org/licenses/by/4.0/
3,bb2666864s,http://creativecommons.org/licenses/by/4.0/
12,bb8676950x,http://creativecommons.org/licenses/by/4.0/
13,bb0691480b,http://creativecommons.org/licenses/by/4.0/
19,bb34560073,http://creativecommons.org/licenses/by/4.0/
27,bb47487939,http://creativecommons.org/licenses/by-nc-sa/4.0/
31,bb3281205b,http://creativecommons.org/licenses/by/4.0/
39,bb5602691s,http://creativecommons.org/licenses/by/4.0/
50,bb66600042,http://creativecommons.org/licenses/by/4.0/
51,bb0892752c,http://creativecommons.org/licenses/by/4.0/


In [15]:
license_counts = license_ids.groupby('license_licenseURI').size().to_frame()
license_counts['percent'] = license_counts[0]/(ids.nunique()) * 100
license_counts

,0,percent
license_licenseURI,,
http://creativecommons.org/licenses/by-nc-sa/4.0/,1,4.0
http://creativecommons.org/licenses/by/4.0/,23,92.0
https://opensource.org/licenses/MIT,1,4.0


## 3. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Description
**Related function:** `mean_characters`

In [16]:
descriptions = df.UCSDRecordsCrosswalk.description
descriptions

0      Publication abstract: Measurement of ice nucle...
1                                                   None
2                                                   None
3      Abstract:  \nThe American Gut Project (AGP) [1...
4                                                   None
                             ...                        
169                                                 None
170    Complex data (hdf files) for one binding site ...
171    Complex data (hdf files) for another binding s...
172    Complex data (hdf files) for a conformation bi...
173    Complex data (hdf files) for another conformat...
Name: Note_description, Length: 174, dtype: object

In [17]:
#description is at object level; remove NAs
descriptions = descriptions.dropna()
descriptions

0      Publication abstract: Measurement of ice nucle...
3      Abstract:  \nThe American Gut Project (AGP) [1...
12     This is the second in a series of concept pape...
13     Over the course of two quarters, our Data Scie...
19     Biologists work with a multitude of protein se...
27     This is the capstone project for MAS DSE progr...
31     Capstone Project for Data Science & Engineerin...
39     Brain tissue has long been studied and detaile...
50     This study tests the utility of convolutional ...
51     Using deep learning transformer model BERT to ...
56     We use 20-year-long records from 1996 to 2015 ...
60     Predicting housing prices in San Diego County....
65     Dynamic computed tomography (DCT) refers to re...
67     Describes MPM-based deformation, CT simulation...
68     Contains 157 different deformation sets, each ...
69     Contains 157 different deformation sets, each ...
70     Recommender systems are algorithms that sugges...
77     Advances in volume elect

In [18]:
print(f'{analysis.mean_characters(descriptions)} mean characters')

1097.6363636363637 mean characters


## 4. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Methods
**Related function:** `mean_characters`

## 5. What are the min and max publication dates for each repo?

## How many objects were published each year for each repo?
**Property:** Publication date

In [19]:
publication_dates = df.UCSDRecordsCrosswalk.publication_date
publication_dates

0      2021.0
1        None
2        None
3      2019.0
4        None
        ...  
169      None
170      None
171      None
172      None
173      None
Name: Date_issued, Length: 174, dtype: object

In [20]:
#min and max publication year
publication_dates.min(), publication_dates.max()

(2019.0, 2021.0)

In [21]:
#objects per year
publication_dates_use = publication_dates.value_counts().sort_index()
publication_dates_use = (publication_dates_use)

In [22]:
publication_dates_use.astype('datetime64[ns]').apply(lambda date: date.year).to_frame()

,Date_issued
2019.0,1970
2020.0,1970
2021.0,1970


In [23]:
pub_dates_export_ready = publication_dates_use.reset_index(level=0)
pub_dates_export_ready.columns = ['year', 'count']
pub_dates_export_ready = pub_dates_export_ready.astype(int)
pub_dates_export_ready

,year,count
0,2019,6
1,2020,12
2,2021,7


In [24]:
#add column with name of repo
pub_dates_export_ready['repo'] = 'ucsd'

In [25]:
#export to Figures folder
pub_dates_export_ready.to_csv('..\\..\\Figures\\Figure1\\repository_dates\\ucsd_pub_years.csv')

## 6. What are the unweighted mean, median, and max file sizes among all ingested files?
**Property:** File size
**Related function:** `get_summary_statistics`

We first get the file size attribute using the crosswalk.

In [26]:
file_sizes = df.UCSDRecordsCrosswalk.file_size
file_sizes

0             None
1           1772.0
2      439893640.0
3             None
4           4963.0
          ...     
169         4887.0
170      5429849.0
171      1987385.0
172     61056224.0
173     65016821.0
Name: file_File_size, Length: 174, dtype: object

In [27]:
#remove NA values
file_sizes = file_sizes.dropna()
file_sizes

1           1772.0
2      439893640.0
4           4963.0
5      134125487.0
6       29915111.0
          ...     
169         4887.0
170      5429849.0
171      1987385.0
172     61056224.0
173     65016821.0
Name: file_File_size, Length: 150, dtype: object

In [28]:
#get summary statistics
analysis.get_summary_statistics(file_sizes)

{'mean': 1843320317.5133333, 'median': 19541413.0, 'max': 34786138185.0}

## 7. What are the mean, median, and max number of files per object?
**Property:** URL
**Related function:** `get_summary_statistics`

In [29]:
files = df.UCSDRecordsCrosswalk.url
files

0                                None
1                       README_v2.txt
2                          \tData.zip
3                                None
4                    README_Group1.md
                    ...              
169                         readme.md
170        spike_AllApproved_cut8.tgz
171       spike1_AllApproved_cut8.tgz
172     protease_AllApproved_cut8.tgz
173    protease2_AllApproved_cut8.tgz
Name: file_Name, Length: 174, dtype: object

In [30]:
#since interested in per object, need to group by ID
files_ids = pd.concat([ids, files], axis = 1)
files_ids

,Object_Unique_ID,file_Name
0,bb5023335q,None
1,bb5023335q,README_v2.txt
2,bb5023335q,\tData.zip
3,bb2666864s,None
4,bb2666864s,README_Group1.md
...,...,...
169,bb4749622c,readme.md
170,bb4749622c,spike_AllApproved_cut8.tgz
171,bb4749622c,spike1_AllApproved_cut8.tgz
172,bb4749622c,protease_AllApproved_cut8.tgz


In [31]:
#remove None values
files_ids = files_ids.dropna()
files_ids

,Object_Unique_ID,file_Name
1,bb5023335q,README_v2.txt
2,bb5023335q,\tData.zip
4,bb2666864s,README_Group1.md
5,bb2666864s,DSE-American-Gut-Project-master.zip
6,bb2666864s,american_gut_capstone_dashboard-master.zip
...,...,...
169,bb4749622c,readme.md
170,bb4749622c,spike_AllApproved_cut8.tgz
171,bb4749622c,spike1_AllApproved_cut8.tgz
172,bb4749622c,protease_AllApproved_cut8.tgz


In [32]:
#group by ID to get count of records/components per object
file_ids_counts = files_ids.groupby('Object_Unique_ID').size()
file_ids_counts

Object_Unique_ID
bb0691480b     5
bb0892752c     4
bb09979489     6
bb1507819f     4
bb20197654    33
bb2666864s     8
bb3077072p     6
bb3281205b     7
bb3352917m     8
bb34560073     7
bb4103775q     8
bb4237495g     5
bb47487939     3
bb4749622c     5
bb4782925w     4
bb5021836n     4
bb5023335q     2
bb5534097t     2
bb5602691s    10
bb66600042     1
bb7787014v     5
bb8503744c     6
bb8676950x     1
bb88449405     4
bb94252168     3
dtype: int64

In [33]:
#confirm accuracy - can see that id bb0691480b has 5 file components
files_ids.sort_values('Object_Unique_ID').head(7)

,Object_Unique_ID,file_Name
15,bb0691480b,Final Report - Fire Weather Data Analysis_ Mod...
18,bb0691480b,Project Poster.pptx
17,bb0691480b,FPI Project Data.zip
16,bb0691480b,DSE_MAS_group6_scripts.zip
14,bb0691480b,Readme Text.txt
53,bb0892752c,final_report_DSE_MAS_group7.pdf
52,bb0892752c,DSE_MAS_Group7_README.md


In [34]:
#get summary statistics
analysis.get_summary_statistics(file_ids_counts)

{'mean': 6.04, 'median': 5.0, 'max': 33}

## 8. What are the mean, median, and max total dataset size (summed across all files) per object?
**Property:** Dataset size
**Related function:** `get_summary_statistics`

In [35]:
dataset_sizes = df.UCSDRecordsCrosswalk.dataset_size
dataset_sizes

0             None
1           1772.0
2      439893640.0
3             None
4           4963.0
          ...     
169         4887.0
170      5429849.0
171      1987385.0
172     61056224.0
173     65016821.0
Name: file_File_size, Length: 174, dtype: object

In [36]:
#since interested in per object, need to group by ID
sizes_ids = pd.concat([ids, dataset_sizes], axis = 1)
sizes_ids

,Object_Unique_ID,file_File_size
0,bb5023335q,None
1,bb5023335q,1772.0
2,bb5023335q,439893640.0
3,bb2666864s,None
4,bb2666864s,4963.0
...,...,...
169,bb4749622c,4887.0
170,bb4749622c,5429849.0
171,bb4749622c,1987385.0
172,bb4749622c,61056224.0


In [37]:
#remove None
sizes_ids = sizes_ids.dropna()
sizes_ids

,Object_Unique_ID,file_File_size
1,bb5023335q,1772.0
2,bb5023335q,439893640.0
4,bb2666864s,4963.0
5,bb2666864s,134125487.0
6,bb2666864s,29915111.0
...,...,...
169,bb4749622c,4887.0
170,bb4749622c,5429849.0
171,bb4749622c,1987385.0
172,bb4749622c,61056224.0


In [38]:
#sum up file size within object
sizes_ids_total = sizes_ids.groupby('Object_Unique_ID').sum()
sizes_ids_total

,file_File_size
Object_Unique_ID,
bb0691480b,1.031012e+08
bb0892752c,3.368242e+08
bb09979489,4.352117e+08
bb1507819f,1.011540e+10
bb20197654,1.308555e+11
bb2666864s,2.895955e+08
bb3077072p,1.350990e+10
bb3281205b,1.385614e+10
bb3352917m,1.475227e+09


In [39]:
#confirm accuracy - bb0691480b has 5 components
sizes_ids.sort_values('Object_Unique_ID').head(7)

,Object_Unique_ID,file_File_size
15,bb0691480b,7928613.0
18,bb0691480b,692676.0
17,bb0691480b,94406318.0
16,bb0691480b,70741.0
14,bb0691480b,2895.0
53,bb0892752c,4847916.0
52,bb0892752c,2449.0


In [40]:
sizes_ids.sort_values('Object_Unique_ID').head(5)['file_File_size'].sum()

103101243.0

In [41]:
sizes_ids_total.head(1)

,file_File_size
Object_Unique_ID,
bb0691480b,103101243.0


In [42]:
#get summary statistics
analysis.get_summary_statistics(sizes_ids_total['file_File_size'])

{'mean': 11059921905.08, 'median': 991279633.0, 'max': 130855475113.0}

## 9. How many of each scientific domain are assigned?
**Property:** Domain
**Related function:** `domains.value_counts()`

In [43]:
domains = df.UCSDRecordsCrosswalk.domain
domains

In [44]:
#confirm missing for this repo
print(df.UCSDRecordsCrosswalk.domain)

None


## 10. What is the mean number of characters (excluding whitespaces, if possible) for usage notes per object?
**Property:** Technical details
**Related function:** `mean_characters`

In [45]:
# "usage notes" is not in crosswalk - need to return to

## 11-13. What are the mean and median total number of keyword terms per object, after merging results for Keyword, Geographic keyword, and Scientific keyword?
**Property:** Keyword

In [46]:
print(df.UCSDRecordsCrosswalk.keyword)

0      High throughput @ bb3112054r|Ice nucleating pa...
1                                                   None
2                                                   None
3      Luigi (Python package) @ bb0619060v|Data Scien...
4                                                   None
                             ...                        
169                                                 None
170                                                 None
171                                                 None
172                                                 None
173                                                 None
Name: Subject_topic, Length: 174, dtype: object


In [47]:
print(df.UCSDRecordsCrosswalk.geographic_keyword)

0      None
1      None
2      None
3      None
4      None
       ... 
169    None
170    None
171    None
172    None
173    None
Name: Subject_geographic, Length: 174, dtype: object


In [48]:
print(df.UCSDRecordsCrosswalk.scientific_keyword)

0      None
1      None
2      None
3      None
4      None
       ... 
169    None
170    None
171    None
172    None
173    None
Name: Subject_scientific_name, Length: 174, dtype: object


In [49]:
#create and concatenate keywords
keywords1 = df.UCSDRecordsCrosswalk.keyword
keywords2 = df.UCSDRecordsCrosswalk.geographic_keyword
keywords3 = df.UCSDRecordsCrosswalk.scientific_keyword

In [50]:
#for keywords1, convert to list and specify | as item separator
keywords1 = keywords1.str.split("|")
keywords1

0      [High throughput @ bb3112054r, Ice nucleating ...
1                                                   None
2                                                   None
3      [Luigi (Python package) @ bb0619060v, Data Sci...
4                                                   None
                             ...                        
169                                                 None
170                                                 None
171                                                 None
172                                                 None
173                                                 None
Name: Subject_topic, Length: 174, dtype: object

In [51]:
#confirm geographic and scientific keywords are not all NA
print(keywords2.unique())
print(keywords3.unique())

[None 'San Diego County @ bb1780134h']
[None 'Drosophila melanogaster @ bb90466558']


In [52]:
#for keywords2, convert to list and specify | as item separator
keywords2 = keywords2.str.split("|")
keywords2

0      None
1      None
2      None
3      None
4      None
       ... 
169    None
170    None
171    None
172    None
173    None
Name: Subject_geographic, Length: 174, dtype: object

In [53]:
#for keywords3, convert to list and specify | as item separator
keywords3 = keywords3.str.split("|")
keywords3

0      None
1      None
2      None
3      None
4      None
       ... 
169    None
170    None
171    None
172    None
173    None
Name: Subject_scientific_name, Length: 174, dtype: object

In [54]:
#concatenate keywords and ids
keywords_all = pd.concat([keywords1, keywords2], axis = 1)
keywords_all = pd.concat([keywords_all, keywords3], axis = 1)
keywords_all = pd.concat([ids, keywords_all], axis = 1)
keywords_all

,Object_Unique_ID,Subject_topic,Subject_geographic,Subject_scientific_name
0,bb5023335q,"[High throughput @ bb3112054r, Ice nucleating ...",None,None
1,bb5023335q,None,None,None
2,bb5023335q,None,None,None
3,bb2666864s,"[Luigi (Python package) @ bb0619060v, Data Sci...",None,None
4,bb2666864s,None,None,None
...,...,...,...,...
169,bb4749622c,None,None,None
170,bb4749622c,None,None,None
171,bb4749622c,None,None,None
172,bb4749622c,None,None,None


In [55]:
#replace the None values with empty lists so the count of string values evaluates to 0
keywords_use = keywords_all.apply(
    lambda row: row.apply(
        lambda cell: cell if cell else []
    ),
    axis=1
)
keywords_use

,Object_Unique_ID,Subject_topic,Subject_geographic,Subject_scientific_name
0,bb5023335q,"[High throughput @ bb3112054r, Ice nucleating ...",[],[]
1,bb5023335q,[],[],[]
2,bb5023335q,[],[],[]
3,bb2666864s,"[Luigi (Python package) @ bb0619060v, Data Sci...",[],[]
4,bb2666864s,[],[],[]
...,...,...,...,...
169,bb4749622c,[],[],[]
170,bb4749622c,[],[],[]
171,bb4749622c,[],[],[]
172,bb4749622c,[],[],[]


In [56]:
#remove ID column
keywords_use = keywords_use.drop(['Object_Unique_ID'], axis = 1)
keywords_use

,Subject_topic,Subject_geographic,Subject_scientific_name
0,"[High throughput @ bb3112054r, Ice nucleating ...",[],[]
1,[],[],[]
2,[],[],[]
3,"[Luigi (Python package) @ bb0619060v, Data Sci...",[],[]
4,[],[],[]
...,...,...,...
169,[],[],[]
170,[],[],[]
171,[],[],[]
172,[],[],[]


In [57]:
#count number of keywords for each row
keyword_counts = keywords_use.apply(
    lambda row: sum([len(row[entry]) for entry in keywords_use.columns if row[entry]]),
    axis=1
)
keyword_counts

0      8
1      0
2      0
3      8
4      0
      ..
169    0
170    0
171    0
172    0
173    0
Length: 174, dtype: int64

In [58]:
#confirm accuracy - first row in dataframe should have 8 keywords and it does
keywords_use.iloc[0][0]

['High throughput @ bb3112054r',
 'Ice nucleating particles @ bb73434075',
 'Snomax @ bb5705934c',
 'Automated detection @ bb0518173k',
 'Machine learning @ bb7001306t',
 'Microfluidic device @ bb8299815r',
 'Deep neural network @ bb7958515k',
 'Polarized light @ bb53646351']

In [59]:
keyword_counts.sort_values(ascending = False)

51     19
31     15
27     13
13     11
70     11
       ..
66      0
67      0
68      0
69      0
173     0
Length: 174, dtype: int64

In [60]:
#check object 51 which says has 19 keywords, which seems high
keywords_use.iloc[51]

Subject_topic              [Neural networks @ bb1131277w, Support Vector ...
Subject_geographic                                                        []
Subject_scientific_name                                                   []
Name: 51, dtype: object

In [61]:
#confirmed accurate
keywords_use.iloc[51][0]

['Neural networks @ bb1131277w',
 'Support Vector Machine (SVM) @ bb9595904w',
 'Jupyter Notebooks @ bb1234051h',
 'Protein analysis @ bb0244283z',
 'Natural Language Processing (NLP) @ bb57392142',
 'SciKit Learn @ bb9903073v',
 'Tensorflow @ bb4749442f',
 'Pytorch @ bb5056610w',
 'Protein embedding @ bb76504923',
 'XGBoost @ bb66607224',
 'Data Science & Engineering Master of Advanced Study (DSE MAS) @ bb1642961z',
 'Language embedding @ bb6080513z',
 'Keras @ bb8674391v',
 'Machine learning @ bb7001306t',
 'Graphics Processing Unit (GPU) @ bb6763112j',
 'Logistic regression @ bb0926882m',
 'Bidirectional Encoder Representations from Transformers (BERT) @ bb51590009',
 'Deep learning @ bb5124223h',
 'Transformers @ bb95617711']

In [62]:
keywords_use.iloc[150][1]

[]

In [63]:
analysis.get_summary_statistics(keyword_counts)

{'mean': 1.1264367816091954, 'median': 0.0, 'max': 19}

## 14. Who are the most common funding agencies for each repo? What are the object counts per agency?
**Property:** Funding Agency

In [64]:
funders = df.UCSDRecordsCrosswalk.funding_agency
funders

0      This work was supported by NSF through the NSF...
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
169                                                 None
170                                                 None
171                                                 None
172                                                 None
173                                                 None
Name: Note_funding, Length: 174, dtype: object

In [65]:
#remove NA values
funders = funders.dropna()
funders

0      This work was supported by NSF through the NSF...
12     The Open Storage Network is funded by the Nati...
50     This study is supported by the U.S. Army Corps...
56     Earth System Grid Federation (ESGF) Project an...
65     Laboratory Directed Research and Development (...
84     NIGMS R35 GM119844 (to K.A.), the Japan Societ...
168    LLNL’s Laboratory Directed Research and Develo...
Name: Note_funding, dtype: object

In [66]:
funders_counts = funders.value_counts().to_frame()
funders_counts

,Note_funding
"This work was supported by NSF through the NSF Center for Aerosol Impacts on Chemistry of the Environment (CAICE), an NSF Funded Center for Chemical Innovation (CHE1801971). Portions of this work were conducted in the Minnesota Nano Center, which is supported by the National Science Foundation through the National Nano Coordinated Infrastructure Network (NNCI) under Award Number ECCS2025124.",1
"The Open Storage Network is funded by the National Science Foundation under grants #1747552, 1747493, 1747507, 1747490, 1747483 and by Schmidt Futures.",1
This study is supported by the U.S. Army Corps of Engineers (USACE)-Cooperative Ecosystem Studies Unit (CESU) as part of Forecast Informed Reservoir Operations (FIRO) under grant W912HZ-15-2-0019 and the California Department of Water Resources Atmospheric River Program under grant number 4600010378 TO#15 Am 22.,1
Earth System Grid Federation (ESGF) Project and Task used for working with this data: 40128 / SCW1480-ESGF.,1
Laboratory Directed Research and Development (LDRD)-FS: 20-FS-010,1
"NIGMS R35 GM119844 (to K.A.), the Japan Society for the Promotion of Science (to K.I.), the Naito Foundation (to K.I.), the Mary K. Chapman Foundation (to M.W.), the Rose Hills Foundation (to M.W.)",1
"LLNL’s Laboratory Directed Research and Development (LDRD), tracking # 20-ERD-065 and 20-ERD-062.\nAHA CRADA: funded by the American Heart Association Center for Accelerated Drug Discovery under a collaborative research and development agreement (CRADA TC02274).",1


## 15. What are the mean, median, and max number of Views per object?
**Property:** Views
**Related function:** `get_summary_statistics`

In [67]:
#done manually

## 16. What are the mean, median, and max (total) number of downloads per object?
**Property:** Downloads
**Related function:** `get_summary_statistics`

In [68]:
#done manually

## 17. What are the mean, median, and max Citation counts per object?
**Property:** Citation count
**Related function:** `get_summary_statistics`

In [69]:
citation_count = df.UCSDRecordsCrosswalk.citation_count
citation_count

In [70]:
#confirm missing for this repo
print(df.UCSDRecordsCrosswalk.citation_count)

None


## 18. How many objects contain each given resource type?
**Property:** Resource type

In [71]:
resource_types = df.UCSDRecordsCrosswalk.resource_type
resource_types

0           data
1           None
2           None
3      data|text
4           None
         ...    
169         None
170         None
171         None
172         None
173         None
Name: Type_of_Resource, Length: 174, dtype: object

In [72]:
#convert to list and specify | as item separator
resource_types_list = resource_types.str.split("|")
resource_types_list

0            [data]
1              None
2              None
3      [data, text]
4              None
           ...     
169            None
170            None
171            None
172            None
173            None
Name: Type_of_Resource, Length: 174, dtype: object

In [73]:
#since interested in object level, need to group by ID
resource_types_ids = pd.concat([ids, resource_types_list], axis = 1)
resource_types_ids

,Object_Unique_ID,Type_of_Resource
0,bb5023335q,[data]
1,bb5023335q,None
2,bb5023335q,None
3,bb2666864s,"[data, text]"
4,bb2666864s,None
...,...,...
169,bb4749622c,None
170,bb4749622c,None
171,bb4749622c,None
172,bb4749622c,None


In [74]:
resource_types_all = resource_types_ids.explode('Type_of_Resource')
resource_types_all

,Object_Unique_ID,Type_of_Resource
0,bb5023335q,data
1,bb5023335q,None
2,bb5023335q,None
3,bb2666864s,data
3,bb2666864s,text
...,...,...
169,bb4749622c,None
170,bb4749622c,None
171,bb4749622c,None
172,bb4749622c,None


In [75]:
#strip any whitespaces from Type_of_Resource
resource_types_all['Type_of_Resource'] = resource_types_all['Type_of_Resource'].str.strip()
resource_types_all

,Object_Unique_ID,Type_of_Resource
0,bb5023335q,data
1,bb5023335q,None
2,bb5023335q,None
3,bb2666864s,data
3,bb2666864s,text
...,...,...
169,bb4749622c,None
170,bb4749622c,None
171,bb4749622c,None
172,bb4749622c,None


In [76]:
#drop None and duplicates
resource_types_all = resource_types_all.drop_duplicates()
resource_types_all = resource_types_all.dropna()
resource_types_all

,Object_Unique_ID,Type_of_Resource
0,bb5023335q,data
3,bb2666864s,data
3,bb2666864s,text
12,bb8676950x,text
13,bb0691480b,data
13,bb0691480b,text
19,bb34560073,data
19,bb34560073,text
27,bb47487939,data
27,bb47487939,text


In [77]:
resource_types_perc = resource_types_all.groupby('Type_of_Resource').count()
resource_types_perc['percent'] = resource_types_perc['Object_Unique_ID']/ids.nunique() * 100
resource_types_perc

,Object_Unique_ID,percent
Type_of_Resource,,
data,24,96.0
software,6,24.0
still image,2,8.0
text,22,88.0


## 19. How many objects contain each type of file extension given?
**Property:** File Extension
**Related function:** `get_file_extensions`

In [78]:
files = df.UCSDRecordsCrosswalk.file_extension
files

0                                None
1                       README_v2.txt
2                          \tData.zip
3                                None
4                    README_Group1.md
                    ...              
169                         readme.md
170        spike_AllApproved_cut8.tgz
171       spike1_AllApproved_cut8.tgz
172     protease_AllApproved_cut8.tgz
173    protease2_AllApproved_cut8.tgz
Name: file_Name, Length: 174, dtype: object

In [79]:
#make as list (should only be one file per row, but confirms)
#for keywords1, convert to list and specify | as item separator
files = files.str.split(",")
files

0                                  None
1                       [README_v2.txt]
2                          [\tData.zip]
3                                  None
4                    [README_Group1.md]
                     ...               
169                         [readme.md]
170        [spike_AllApproved_cut8.tgz]
171       [spike1_AllApproved_cut8.tgz]
172     [protease_AllApproved_cut8.tgz]
173    [protease2_AllApproved_cut8.tgz]
Name: file_Name, Length: 174, dtype: object

In [80]:
#add ID column to make easier to count by object
files_ids = pd.concat([ids, files], axis = 1)
files_ids

,Object_Unique_ID,file_Name
0,bb5023335q,None
1,bb5023335q,[README_v2.txt]
2,bb5023335q,[\tData.zip]
3,bb2666864s,None
4,bb2666864s,[README_Group1.md]
...,...,...
169,bb4749622c,[readme.md]
170,bb4749622c,[spike_AllApproved_cut8.tgz]
171,bb4749622c,[spike1_AllApproved_cut8.tgz]
172,bb4749622c,[protease_AllApproved_cut8.tgz]


In [81]:
#remove None
files_ids = files_ids.dropna()
files_ids

,Object_Unique_ID,file_Name
1,bb5023335q,[README_v2.txt]
2,bb5023335q,[\tData.zip]
4,bb2666864s,[README_Group1.md]
5,bb2666864s,[DSE-American-Gut-Project-master.zip]
6,bb2666864s,[american_gut_capstone_dashboard-master.zip]
...,...,...
169,bb4749622c,[readme.md]
170,bb4749622c,[spike_AllApproved_cut8.tgz]
171,bb4749622c,[spike1_AllApproved_cut8.tgz]
172,bb4749622c,[protease_AllApproved_cut8.tgz]


The following code separates the full file extensions (all dot-separated values after the first dot) for a list of files and creates a set, allowing us to only look at the number of objects that contain a given extension.

In [82]:
files_extension_set = files_ids['file_Name'].apply(
    lambda file_list: list({''.join(Path(file).suffixes) for file in file_list})
)
files_extension_set

1      [.txt]
2      [.zip]
4       [.md]
5      [.zip]
6      [.zip]
        ...  
169     [.md]
170    [.tgz]
171    [.tgz]
172    [.tgz]
173    [.tgz]
Name: file_Name, Length: 151, dtype: object

In [83]:
#add ID column to make easier to count by object
ext_object = pd.concat([ids, files_extension_set], axis = 1)
ext_object

,Object_Unique_ID,file_Name
0,bb5023335q,NaN
1,bb5023335q,[.txt]
2,bb5023335q,[.zip]
3,bb2666864s,NaN
4,bb2666864s,[.md]
...,...,...
169,bb4749622c,[.md]
170,bb4749622c,[.tgz]
171,bb4749622c,[.tgz]
172,bb4749622c,[.tgz]


The following code separates the full file extensions (all dot-separated values after the first dot) for a list of files and creates a set, allowing us to only look at the number of objects that contain a given extension.

In [84]:
#separate out any multi-item lists and convert to non-list
#drop any induced NA values
ext_object_use = ext_object.explode('file_Name')
ext_object_use = ext_object_use.dropna()
ext_object_use

,Object_Unique_ID,file_Name
1,bb5023335q,.txt
2,bb5023335q,.zip
4,bb2666864s,.md
5,bb2666864s,.zip
6,bb2666864s,.zip
...,...,...
169,bb4749622c,.md
170,bb4749622c,.tgz
171,bb4749622c,.tgz
172,bb4749622c,.tgz


In [85]:
#drop duplicates, so ext type per object
ext_object_use_unique = ext_object_use.drop_duplicates()
ext_object_use_unique

,Object_Unique_ID,file_Name
1,bb5023335q,.txt
2,bb5023335q,.zip
4,bb2666864s,.md
5,bb2666864s,.zip
7,bb2666864s,.pdf
9,bb2666864s,
12,bb8676950x,.pdf
14,bb0691480b,.txt
15,bb0691480b,.docx
16,bb0691480b,.zip


In [86]:
#group by ext type and count objects
ext_counts = ext_object_use_unique.groupby('file_Name').size().to_frame().sort_values(0, ascending = False)
ext_counts['percent'] = round(ext_counts[0]/ids.nunique()*100)
ext_counts

,0,percent
file_Name,,
.zip,21,84.0
.pdf,16,64.0
.md,13,52.0
.txt,8,32.0
.tgz,3,12.0
,2,8.0
.png,2,8.0
.PDF,1,4.0
.docx,1,4.0


In [87]:
#export for further clean up, refining estimates, and plotting

In [88]:
#reset index and update column names
ext_grouped_ready = ext_object_use_unique.copy(deep = True)
ext_grouped_ready.columns = ['index', 'files']

#add column with name of repo
ext_grouped_ready['repo'] = 'ucsd'

ext_grouped_ready

,index,files,repo
1,bb5023335q,.txt,ucsd
2,bb5023335q,.zip,ucsd
4,bb2666864s,.md,ucsd
5,bb2666864s,.zip,ucsd
7,bb2666864s,.pdf,ucsd
9,bb2666864s,,ucsd
12,bb8676950x,.pdf,ucsd
14,bb0691480b,.txt,ucsd
15,bb0691480b,.docx,ucsd
16,bb0691480b,.zip,ucsd


In [89]:
#export to Figures folder
ext_grouped_ready.to_csv('..\\..\\Figures\\Figure2\\file_ext_data\\ucsd_extensions.csv')

## 19.5 How many files of each type of file extension are present?
**Property:** File extension

In [90]:
#start from files_extension_set
files_extension_set

1      [.txt]
2      [.zip]
4       [.md]
5      [.zip]
6      [.zip]
        ...  
169     [.md]
170    [.tgz]
171    [.tgz]
172    [.tgz]
173    [.tgz]
Name: file_Name, Length: 151, dtype: object

In [91]:
#expand any multi-item lists
ext_all = files_extension_set.explode()
ext_all

1      .txt
2      .zip
4       .md
5      .zip
6      .zip
       ... 
169     .md
170    .tgz
171    .tgz
172    .tgz
173    .tgz
Name: file_Name, Length: 151, dtype: object

In [92]:
#count up by file extension
ext_all.value_counts().head(10)

.zip       80
.pdf       29
.md        13
.txt        8
.tgz        7
            2
.png        2
.twbx       2
.tar.gz     2
.docx       1
Name: file_Name, dtype: int64

## 20. How many objects contain each type of File format given?
**Property:** File format

In [93]:
file_formats = df.UCSDRecordsCrosswalk.file_format
file_formats

0             None
1       Plain text
2       ZIP Format
3             None
4       Plain text
          ...     
169     Plain text
170    GZIP Format
171    GZIP Format
172    GZIP Format
173    GZIP Format
Name: file_File_format, Length: 174, dtype: object

In [94]:
#since interested in object level, need to group by ID
file_formats_ids = pd.concat([ids, file_formats], axis = 1)
file_formats_ids

,Object_Unique_ID,file_File_format
0,bb5023335q,None
1,bb5023335q,Plain text
2,bb5023335q,ZIP Format
3,bb2666864s,None
4,bb2666864s,Plain text
...,...,...
169,bb4749622c,Plain text
170,bb4749622c,GZIP Format
171,bb4749622c,GZIP Format
172,bb4749622c,GZIP Format


In [95]:
#drop duplicates
file_formats_ids = file_formats_ids.drop_duplicates()
file_formats_ids

,Object_Unique_ID,file_File_format
0,bb5023335q,None
1,bb5023335q,Plain text
2,bb5023335q,ZIP Format
3,bb2666864s,None
4,bb2666864s,Plain text
5,bb2666864s,ZIP Format
7,bb2666864s,Portable Document Format
12,bb8676950x,Portable Document Format
13,bb0691480b,None
14,bb0691480b,Plain text


In [96]:
#group by format type to count how many objects have each format type
formats_grouped = file_formats_ids.groupby('file_File_format').count().sort_values('Object_Unique_ID', ascending = False)
formats_grouped['percent'] = formats_grouped['Object_Unique_ID']/ids.nunique() * 100
formats_grouped

,Object_Unique_ID,percent
file_File_format,,
Plain text,21,84.0
ZIP Format,21,84.0
Portable Document Format,17,68.0
Extensible Markup Language,2,8.0
GZIP Format,2,8.0
Portable Network Graphics,2,8.0
application/x-gzip,2,8.0
Office Open XML Document,1,4.0
PPTX,1,4.0


## 21. How many objects contain each type of Media type given?
**Property:** Media type

In [97]:
media_types = df.UCSDRecordsCrosswalk.media_type
media_types

0                    None
1            \ttext/plain
2         application/zip
3                    None
4              text/plain
              ...        
169          \ttext/plain
170    application/x-gzip
171    application/x-gzip
172    application/x-gzip
173    application/x-gzip
Name: file_Media_type, Length: 174, dtype: object

In [98]:
#since interested in object level, need to group by ID
media_types_ids = pd.concat([ids, media_types], axis = 1)
media_types_ids

,Object_Unique_ID,file_Media_type
0,bb5023335q,None
1,bb5023335q,\ttext/plain
2,bb5023335q,application/zip
3,bb2666864s,None
4,bb2666864s,text/plain
...,...,...
169,bb4749622c,\ttext/plain
170,bb4749622c,application/x-gzip
171,bb4749622c,application/x-gzip
172,bb4749622c,application/x-gzip


In [99]:
#drop duplicates
media_types_ids = media_types_ids.drop_duplicates()
media_types_ids

,Object_Unique_ID,file_Media_type
0,bb5023335q,None
1,bb5023335q,\ttext/plain
2,bb5023335q,application/zip
3,bb2666864s,None
4,bb2666864s,text/plain
5,bb2666864s,application/zip
7,bb2666864s,application/pdf
12,bb8676950x,application/pdf
13,bb0691480b,None
14,bb0691480b,text/plain


In [100]:
#group by media type to count how many objects have each media type
media_grouped = media_types_ids.groupby('file_Media_type').count().sort_values('Object_Unique_ID', ascending = False)
media_grouped['percent'] = media_grouped['Object_Unique_ID']/ids.nunique() * 100
media_grouped

,Object_Unique_ID,percent
file_Media_type,,
application/zip,21,84.0
\ttext/plain,19,76.0
application/pdf,17,68.0
GZIP Format,2,8.0
application/x-gzip,2,8.0
image/png,2,8.0
text/plain,2,8.0
text/xml,2,8.0
\tapplication/vnd.openxmlformats-officedocument.presentationml.presentation,1,4.0


## 22. a) How many objects report one related resource type, and b) how many objects report each of those types? c) How many objects report multiple related resource types (regardless of which types)?
**Property:** Related resource type

In [101]:
#related resources done manually for this repo


## 23-25. If there is an entry for an object in one of the three properties (Original data URL, Primary manuscript PID/URL, and Related resource identifier) count as Related resources = True and then count the number of objects that return True.
**Property:** Related Resource Identifier

In [102]:
#done manually for this repo

## 23-25. Also, what is the mean number of related resource links per object (again looking at the three properties (Original data URL, Primary manuscript PID/URL, nd Related resource identifier)?
**Property:** Related Resource Identifier

## 26. How many objects report each relation type? How many objects report multiple relation types, regardless of what those types are?
**Property:** Related resource relation type

In [103]:
#done manually for this repo

## 27. For repositories that store the full citation in a designated field, how many objects have a populated citation? How many objects have a citation and a URL or other actionable link?
**Property:** Citation

In [104]:
#done manually for this repo